<div style="width:100%; text-align:center; margin: 40px 0;">
    <h1 style="color: #003366; font-size: 40px; font-weight:bold; text-shadow: 2px 2px 5px grey;">
        Énergie Électrique en France
    </h1>
    <hr style="width: 50%; height: 3px; background-color: #003366; border: none;">
    <p style="font-style:italic; font-size:18px; color: #555;">
        Une exploration approfondie de l'énergie électrique en France
    </p>
</div>


<h2 style="padding:16px; color:#FFF; background:#07D">Imports</h2>

In [8]:
# Data
URL = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/"

# Imports
import json
import requests
import time
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from ipywidgets import interact, IntSlider, VBox, HBox, DatePicker
from IPython.display import display

<h2 style="padding:16px; color:#FFF; background:#07D">I. Présentation des données</h2>

éCO2mix est un outil créé par RTE France afin de mieux connaître en temps réel l'électricité en France. RTE France met à disposition gratuitement, via plusieurs API, les base de données utilisée par éCO2mix qui fournissent les données de 2013 à aujourd'hui. Les informations fournient par les API sont les suivantes :

- Consommation d’électricité en France métropolitaine (hors Corse) et prévisions calculées la vieille et en début d’après-midi le jour J ;
- Production d’électricité française par filière en valeur et en pourcentage ;
- Échanges commerciaux d’électricité avec les 6 pays voisins ;
- Émissions de CO2 que génère la production d’électricité ;
- Données mensuelles à l’échelle régionale : consommation, production et flux interrégionaux.

Les données sont séparées en 4 bases de données :

- Données éCO2mix nationales temps réel
- Données éCO2mix nationales consolidées et définitives
- Données éCO2mix régionales temps réel
- Données éCO2mix régionales consolidées et définitives

<h2 style="padding:16px; color:#FFF; background:#07D">II. Objectif de l'étude</h2>

Cette étude a pour but l'analyse de la production, de la consommation et des échanges d'électricité en France depuis 2013. L'objectif est de déterminer les parts des différentes filières dans la production d'électricité et de voir leur évolution au cours du temps, de comprendre les tendances dans la consommation d'électricité sur plusieurs échelles de temps et d'identifier la tendance commerciale de la France avec ses pays voisins sur le marché de l'électricité. Enfin, cette étude nous permettra d'étudier l'impact de la production d'électricité sur les émissions de CO2 en France.

<h2 style="padding:16px; color:#FFF; background:#07D">III. Récupération des données</h2>

In [9]:
def fetch_data_by_date(data:str, start:int, rows:int, date:str):
    """Fetch data from a dataset by date and offset
    
    Parameters
    ----------
    data : str
        Name of the dataset.
    start : int
        Offset.
    rows : int
        Number of rows.
    date : str
        Date of the data.

    Returns
    -------
    dict
        Dictionary containing the data. 
    """
    url = f"{URL}{data}" + "/records"
    params = {
        "offset" : start,
        "rows": rows,
        "where": f"date='{date}'"
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('results', {})
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return 
    
def get_length_per_date(data:str, date:str):
    """Get the number of rows for a given date

    Parameters
    ----------
    data : str
        Name of the dataset.
    date : str
        Date of the data.

    Returns
    -------
    int
        Number of rows.
    """
    url = f"{URL}{data}" + "/records"
    params = {
        "select": "date",
        "rows": 1,
        "where": f"date='{date}'"
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('total_count')
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return
    
def insert_data_into_json(file_name: str, new_data: list):
    """Insère de nouvelles données dans un fichier JSON existant.

    Parameters
    ----------
    file_name : str
        Le nom du fichier JSON.
    new_data : list
        La liste des nouvelles données à insérer.
    """
    try:
        with open(file_name, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []

    existing_data.extend(new_data)

    with open(file_name, 'w') as file:
        json.dump(existing_data, file)

def get_date(data: str, first: bool=True) -> str:
    """Get the minimum or maximum date in a dataset

    Parameters
    ----------
    data : str
        Name of the dataset.
    first : bool, optional
        If True, get the minimum date, else get the maximum date. The default is True.
    
    Returns
    -------
    str
        Date.
    """
    date = "date" if first else "-date"
    
    url = f"{URL}{data}" + "/records"
    params = {
        "select": "date",
        "rows": 1,
        "order_by": date,
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('results')[0]['date']
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return

In [10]:
def get_data(from_data: str, json_name: str) -> None:
    """Get data from a dataset and insert it into a JSON file

    Parameters
    ----------
    from_data : str
        Name of the dataset.
    json_name : str
        Name of the JSON file.
    """
    step = 100 # Number of rows to be retrieved per request (API limit)
    start_date = get_date(from_data)
    end_date = get_date(from_data, first = False)
    current_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    
    while current_date <= end_date:
        formatted_date = str(current_date.strftime('%Y-%m-%d'))
        lines_per_date = get_length_per_date(from_data, str(formatted_date))
        print(formatted_date)
        
        for i in range(0, lines_per_date, step):
            rows = min(step, lines_per_date - i)
            try:
                data = fetch_data_by_date(from_data, i, rows, str(formatted_date))
                insert_data_into_json(json_name, data)
                    
            except Exception as e:
                print(f"Erreur lors de l'insertion des données pour la date {formatted_date}, offset {i}: {e}")
                print("Réessai dans 5 secondes...")
                time.sleep(5)  
                i -= step  
                continue
                
        current_date += datetime.timedelta(days=1)

# get_data("eco2mix-national-tr", "national.json")

In [11]:
def transform_json_to_df(file_name:str):
    """Transform a Json File to a pandas DataFrame"""
    with open(file_name) as f:
        data = json.load(f)
    return pd.DataFrame(data)
    
df = transform_json_to_df('national.json')
df

,perimetre,nature,date,heure,date_heure,consommation,prevision_j1,prevision_j,fioul,charbon,...,gaz_ccg,gaz_autres,hydraulique_fil_eau_eclusee,hydraulique_lacs,hydraulique_step_turbinage,bioenergies_dechets,bioenergies_biomasse,bioenergies_biogaz,stockage_batterie,destockage_batterie
0,France,Données temps réel,2022-06-01,00:00,2022-05-31T22:00:00+00:00,44940.0,44800.0,45100.0,144.0,0.0,...,3168.0,0.0,4331.0,1054.0,1677.0,170.0,584.0,286.0,ND,ND
1,France,Données temps réel,2022-06-01,02:15,2022-06-01T00:15:00+00:00,39778.0,39450.0,39600.0,143.0,6.0,...,2726.0,0.0,4053.0,840.0,272.0,176.0,564.0,276.0,ND,ND
2,France,Données temps réel,2022-06-01,05:15,2022-06-01T03:15:00+00:00,37506.0,37150.0,37200.0,144.0,7.0,...,3270.0,0.0,3851.0,610.0,226.0,159.0,562.0,276.0,ND,ND
3,France,Données temps réel,2022-06-01,07:15,2022-06-01T05:15:00+00:00,44259.0,44650.0,44150.0,144.0,10.0,...,3581.0,0.0,4506.0,1302.0,855.0,163.0,568.0,276.0,ND,ND
4,France,Données temps réel,2022-06-01,07:30,2022-06-01T05:30:00+00:00,45062.0,45700.0,45100.0,144.0,10.0,...,3611.0,0.0,4522.0,1314.0,984.0,160.0,568.0,276.0,ND,ND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49184,France,Données temps réel,2024-01-06,20:00,2024-01-06T19:00:00+00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
49185,France,Données temps réel,2024-01-06,20:30,2024-01-06T19:30:00+00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
49186,France,Données temps réel,2024-01-06,21:30,2024-01-06T20:30:00+00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
49187,France,Données temps réel,2024-01-06,21:45,2024-01-06T20:45:00+00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [12]:
@interact(day=(1,30))
def line_chart_consumption_november(day: int = 1) -> None:
    """Generate an interactive line chart of electricity consumption in France for a specific day in November 2023.

    Parameters
    ----------
    day : int
        Day of November 2023.
    
    Returns
    -------
    None

    """
    df = transform_json_to_df('national.json')
    df = df[df['date'] == f'2023-11-{day:02}']
    df = df.sort_values(by=['heure'])
    
    plt.figure(figsize=(13.5,3))
    plt.plot(df['heure'], df['consommation'])
    plt.xticks([f'{str(i).zfill(2)}:00' for i in range(24)], [f'{i:02}h' for i in range(24)])
    plt.xlim(left='00:00', right='23:45')
    plt.xlabel('Heure')
    plt.ylabel('Consommation')
    plt.title(f'Consommation électrique en France le {day:02} novembre 2023')

interactive(children=(IntSlider(value=1, description='day', max=30, min=1), Output()), _dom_classes=('widget-i…

In [13]:
# Créer le widget pour choisir le jour
day_slider = IntSlider(min=1, max=30, value=1, description='Jour', continuous_update=False)

# Initialiser la figure Plotly
fig = go.FigureWidget()

# Fonction de callback pour mettre à jour la figure
def update_figure(change):
    new_day = day_slider.value
    df = transform_json_to_df('national.json')
    df = df[df['date'] == f'2023-11-{new_day:02}']
    df = df.sort_values(by=['heure'])
    with fig.batch_update():
        fig.data = []
        fig.add_scatter(x=df["heure"], y=df["consommation"], mode='lines')
        fig.layout.title = f'Consommation électrique en France le {new_day:02} novembre 2023'

# Ajouter le callback au widget
day_slider.observe(update_figure, names='value')

# Afficher le widget et la figure initiale
display(VBox([day_slider, fig]))

# Appeler manuellement pour initialiser la figure
update_figure(None)



…

## Consommation

In [14]:
default_date = datetime.strptime("2023-11-01", "%Y-%m-%d").date()

date_picker = DatePicker(description='Date', disabled=False, value=default_date)

fig = go.FigureWidget()

def update_figure_consommation(change):
    new_date = date_picker.value
    df = transform_json_to_df('national.json')
    df = df[df['date'] == f'{new_date}']
    df = df.sort_values(by=['heure'])
    with fig.batch_update():
        fig.data = []
        fig.add_scatter(x=df["heure"], y=df["consommation"], mode='lines', fill='tozeroy', name='Consommation')
        fig.add_scatter(x=df["heure"], y=df["prevision_j"], mode='lines', name='Prévision J')
        fig.add_scatter(x=df["heure"], y=df["prevision_j1"], mode='lines', name='Prévision J-1')
        fig.layout.title = f'Consommation électrique en France le {new_date}'
    

date_picker.observe(update_figure_consommation, names='value')

display(VBox([date_picker, fig]))

update_figure_consommation(None)

    'd…